CARNOT equation without diffuse radiation:
$$
\begin{equation}
    \begin{split}
        \dot m C_p \frac{(T_{out} - T_{in})}{A} & = F'\tau \alpha K_{dir} I_{dir} + c_1 (T_m - T_{amb}) \\
        & - c_2 (T_m - T_{amb})^2 - c_3 v_w (T_m - T_{amb}) - c_6 v_w I_{glb} \\
        & + c_4 [E_{longwave} - \sigma_{SB} (T_{amb} + 273.15)^4] - c_5 \frac{dT_m}{dt}
    \end{split}
\end{equation}
$$




All temperatures are reported in Kelvin. Substituting $T_{out} = 2T_m -Tin$ and solving for $\frac{dT_m}{dt}$ we get:

$$
\begin{equation}
    \begin{split}
         \frac{dT_m}{dt} & = \frac{1}{c_5} [F'\tau \alpha K_{dir} I_{dir} + F'\tau \alpha + c_1 (T_m - T_{amb}) \\
        & - c_2 (T_m - T_{amb})^2 - c_3 v_w (T_m - T_{amb}) - c_6 v_w I_{glb} \\
        & + c_4 (E_{longwave} - \sigma_{SB} T_{amb}^4) -2 \frac{\dot m C_p}{A} (T_{m} - T_{in})]
    \end{split}
\end{equation}
$$

In [18]:
# TODO write interpolation function for K_dir = f(theta_transv, theta_long)
# CARNOT parameters
t_0 = 20 + 273  # Initial temperature
A = 1           # surface  of one collector module in m2
F_ta = 0.507    # Fin Efficiency * Transmission * Absorption (F'(tau alfa))
c1 = 0.7153     # heat loss coefficient at (Tm-Ta)=0  [W/(m²*K)]
c2 = 0.00339    # temperature dependence of the heat loss coefficient [W/(m*K)²]
c3 = 0          # wind speed dependence of the heat loss coefficient [J/(m³K)]
c4 = 0          # sky temperature dependence of the heat loss coefficient [W/(m²K)]
c5 = 675        # effective thermal capacity [J/(m²K)]
c6 = 0          # wind dependence in the zero loss efficiency [s/m]
kb = [0, 10, 20, 30, 40, 50, 60, 70, 90]    # angles for longitudinal and transversal IAM [degree]
kbL = [1, 1, 1, 0.99, 0.98, 0.96, 0.93, 0.87, 0]    #values for longitudinal incidence angle modifier (0..1)
kbT = [1, 1.01, 1.04, 1.07, 1.09, 1.14, 1.22, 1.24, 0]  # values for transversal incidence angle modifier (0..1)
kd = 1  # Incidence angle modifier diffuse radiation

# my parameters
mdot = 80 / 3.6e+6  # mass flow rate (L/h to m3/s)
Idir = 1000         # W/m2
t_amb = 300         # ambient temperature
t_in = 290          # water inlet temperature
cp = 4200           # heat capacity (J/kg*k)

For our test case, $c_3 = c_2 = c_4 = c_6 = 0$:

$$
\begin{equation}
    \begin{split}
         \frac{dT_m}{dt} & = \frac{1}{c_5} [F'\tau \alpha K_{dir} I_{dir} + F'\tau \alpha + c_1 (T_m - T_{amb}) - c_2 (T_m - T_{amb})^2 -2 \frac{\dot m C_p}{A} (T_{m} - T_{in})]
    \end{split}
\end{equation}
$$

$K_{dir} = K_b(L) \cdot K_b(T)$

In [20]:
import numpy as np
from scipy.interpolate import interp1d

angles = kb
kbT_spline = interp1d(angles, kbT, kind='cubic')
kbL_spline = interp1d(angles, kbL, kind='cubic')


def kbt(angle_transv):
    return kbT_spline(angle_transv)


def kbl(angle_long):
    return kbL_spline(angle_long)


def kdir(angle_long, angle_transv):
    return kbl(angle_long)*kbt(angle_transv)


def model(Tm, t):
    dTmdt = (F_ta*kdir(0, 0)*Idir + F_ta + c1 * (Tm - t_amb) - c2 * (Tm - t_amb)**2 - 2 *mdot*cp*(Tm - t_in)/A)/c5
    return dTmdt


1.0
